In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
plt.rcParams['mathtext.default']='regular'
import cmocean
import matplotlib.gridspec as gridspec
from cartopy.util import add_cyclic_point
import numpy as np
import xarray as xr

from config_calc import *
import proc_cesm_le
import plot_tools


tr85 = proc_cesm_le.open_ens('tr85','ann_dft',['TEMP','PD'])
tr45 = proc_cesm_le.open_ens('tr45','ann_dft',['TEMP','PD'])

tr85

<xarray.Dataset>
Dimensions:               (d2: 2, ens: 33, lat_aux_grid: 395, moc_comp: 3, moc_z: 61, nlat: 384, nlon: 320, time: 182, transport_comp: 5, transport_reg: 2, z_t: 60, z_t_150m: 15, z_w: 60, z_w_bot: 60, z_w_top: 60)
Coordinates:
  * time                  (time) float64 7.01e+05 7.01e+05 7.014e+05 ...
  * z_t                   (z_t) float64 500.0 1.5e+03 2.5e+03 3.5e+03 ...
  * lat_aux_grid          (lat_aux_grid) float32 -79.4882 -78.9529 -78.4187 ...
  * moc_z                 (moc_z) float32 0.0 1000.0 2000.0 3000.0 4000.0 ...
  * z_t_150m              (z_t_150m) float32 500.0 1500.0 2500.0 3500.0 ...
  * z_w                   (z_w) float32 0.0 1000.0 2000.0 3000.0 4000.0 ...
  * z_w_bot               (z_w_bot) float32 1000.0 2000.0 3000.0 4000.0 ...
  * z_w_top               (z_w_top) float32 0.0 1000.0 2000.0 3000.0 4000.0 ...
Dimensions without coordinates: d2, ens, moc_comp, nlat, nlon, transport_comp, transport_reg
Data variables:
    ANGLE                 (ens, nl

In [ ]:
tr85a = tr85.mean(dim='ens')
tr45a = tr45.mean(dim='ens')
tr85a


In [ ]:
#reload(plot_tools)
cmap = cmocean.cm.balance #'Blues_r'
units = '$^\circ$C'
levels = np.arange(-2.,2.1,0.1)
print levels

#matplotlib.rcParams['contour.negative_linestyle'] = 'solid'

plot_name = os.path.join(diro['fig'],'cesm-temperature-change-100m.pdf')

fig=plt.figure(figsize=(10, 8),dpi=300)
gs = gridspec.GridSpec(2,1)

norm = plot_tools.MidPointNorm(midpoint=0.0)
contour_specs = {'levels':levels,'cmap':cmap,'extend':'max','norm':norm}

baseline = tr85a.TEMP[0:20,0,:,:].mean(dim='time')
print baseline.shape

ds = tr85a.isel(time=-1,z_t=0)
print ds
field = ds.TEMP.values
print field.shape
exit()
field = field - baseline

lon,lat,z= plot_tools.adjust_pop_grid(tr85.TLONG.values,tr85.TLAT.values,field)
        
obj = plot_tools.canvas_map_contour_overlay(lon,lat,z,
                                            contour_specs,
                                            units = units,
                                            fig=fig,
                                            gridspec=gs,
                                            row=0,col=0)
plt.savefig(plot_name,bbox_inches='tight')

[ -2.00000000e+00  -1.90000000e+00  -1.80000000e+00  -1.70000000e+00
  -1.60000000e+00  -1.50000000e+00  -1.40000000e+00  -1.30000000e+00
  -1.20000000e+00  -1.10000000e+00  -1.00000000e+00  -9.00000000e-01
  -8.00000000e-01  -7.00000000e-01  -6.00000000e-01  -5.00000000e-01
  -4.00000000e-01  -3.00000000e-01  -2.00000000e-01  -1.00000000e-01
   1.77635684e-15   1.00000000e-01   2.00000000e-01   3.00000000e-01
   4.00000000e-01   5.00000000e-01   6.00000000e-01   7.00000000e-01
   8.00000000e-01   9.00000000e-01   1.00000000e+00   1.10000000e+00
   1.20000000e+00   1.30000000e+00   1.40000000e+00   1.50000000e+00
   1.60000000e+00   1.70000000e+00   1.80000000e+00   1.90000000e+00
   2.00000000e+00]
(384, 320)


In [ ]:
u = tr85a.sel(z_t=slice(0,50e2)).mean(dim='z_t')
l = tr85a.sel(z_t=slice(100e2,200e2)).mean(dim='z_t')
print u
print l